In [1]:
import re
import codecs

In [2]:
txt = codecs.open('./data/rank/rank.txt',"r","utf-8").read()

In [3]:
exec(open("preppy.py").read())

In [4]:
split_reg = re.compile('\r\n')

# parse the raw text of QA into dictionary

In [5]:
books = split_reg.split(txt)

In [6]:
print(books[0])

11	0	那 我 也是 曾经 的 情人 啊 最 伤 自尊 的 是 他 姐姐 说 她 看到 我 时 一度 质疑 我 ex 的 审美 不过 那 时候 我 还是 高中生	耐心 的 双鱼 还好 点 但是 八 竿子 打 不 出来 一句话 那 就 悲剧 了


One single line demo

In [7]:
verses_splitter = re.compile('(?P<user>\d+)\t(?P<label>\d+)\t(?P<query>.+?)\t(?P<response>.+)',)

In [8]:
gen = verses_splitter.finditer(books[0].replace("\n",""))

In [9]:
next(gen).groupdict()

{'user': '11',
 'label': '0',
 'query': '那 我 也是 曾经 的 情人 啊 最 伤 自尊 的 是 他 姐姐 说 她 看到 我 时 一度 质疑 我 ex 的 审美 不过 那 时候 我 还是 高中生',
 'response': '耐心 的 双鱼 还好 点 但是 八 竿子 打 不 出来 一句话 那 就 悲剧 了'}

Lets run all of that on the entire text

In [10]:
examples= []
for line in books:
    for verse_regex_match in verses_splitter.finditer(line):
        example = verse_regex_match.groupdict()
        example.update({"user":example["user"],"label":example["label"],"query":example["query"].strip(),"response":example["response"].strip()})
        examples.append(example)

In [11]:
len(examples)
print((examples[0]))

{'user': '11', 'label': '0', 'query': '那 我 也是 曾经 的 情人 啊 最 伤 自尊 的 是 他 姐姐 说 她 看到 我 时 一度 质疑 我 ex 的 审美 不过 那 时候 我 还是 高中生', 'response': '耐心 的 双鱼 还好 点 但是 八 竿子 打 不 出来 一句话 那 就 悲剧 了'}


In [12]:
import pickle
pickle.dump(examples,open('./data/rank/examples.pkl','wb'))

# Now we make them into TF records

In [13]:
import tensorflow as tf
from preppy import MyPreppy
import numpy as np

In [14]:
import random
random.shuffle(examples)
val_size = int(len(examples)*0.25)
val,train = examples[:val_size], examples[val_size:]

In [15]:
def tokenizer(sentence):
    return sentence.split()
    
BP =MyPreppy(tokenizer_fn=tokenizer) #Charecter level tokenization
for (data,path) in [(val,'./data/rank/val.tfrecord'),(train,'./data/rank/train.tfrecord')]:
    with open(path,'w') as f:
        writer = tf.python_io.TFRecordWriter(f.name)
    for example in data:
        record = BP.sequence_to_tf_example(sequence_q=example["query"],
                                           sequence_a=example["response"],
                                           label=int(example["label"]),
                                           user = int(example["user"]))
        writer.write(record.SerializeToString())


In [16]:
BP.update_reverse_vocab()

In [17]:
pickle.dump(BP,open('./data/rank/preppy.pkl','wb'))

In [18]:
len(BP.vocab)

17038

In [19]:
BP.vocab["<START>"]

1

In [20]:
BP.vocab

defaultdict(<bound method Preppy.next_value of <preppy.MyPreppy object at 0x0000027A5ECE8CF8>>,
            {'<PAD>': 0,
             '<START>': 1,
             '<EOS>': 2,
             '说': 3,
             '实话': 4,
             '泰晤士小镇': 5,
             '那里': 6,
             '房子': 7,
             '都是': 8,
             '别墅': 9,
             '呀': 10,
             '自己': 11,
             '租': 12,
             '的': 13,
             '感觉': 14,
             '很少': 15,
             '而且': 16,
             '除非': 17,
             '你': 18,
             '开车': 19,
             '否则': 20,
             '离': 21,
             '地铁站': 22,
             '很': 23,
             '远': 24,
             '还要': 25,
             '坐': 26,
             '公交': 27,
             '转车': 28,
             '不过': 29,
             '周末': 30,
             '过去': 31,
             '转转': 32,
             '拍拍': 33,
             '照': 34,
             '还是': 35,
             '不错': 36,
             '3': 37,
             '个': 38,
             '

In [21]:
voca_file = codecs.open("./data/voca.txt","w","utf-8")
voca_file.write(str(BP.vocab))
voca_file.close()